### 1시간 봉 데이터 사용

* 시장수익률을 넘는 경우는 전체 10개 중 1개밖에 없었다(@blockchain)
* 생각나는 방안 : 여러 트윗데이터 합치기, 1시간봉에서 바꾸기

In [3]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
import seaborn as sns
from datetime import datetime
import pyupbit
from matplotlib import rcParams

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\choi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [14]:
tweet_list = ["binance", "justinsuntron", "rogerkver", "cz_binance", "brian_armstrong", "NickSzabo4", "ethereum", 
             "blockchain", "BitcoinMagazine", "CoinDesk"]

In [15]:
def sentiment_analysis(tw_group):
    zxcv = max(tw_group['pos'], tw_group['neg'])
    if tw_group['pos'] == zxcv:
        return 'pos'
#     elif tw_group['neu'] == zxcv:
#         return 'neu'
    elif tw_group['neg'] == zxcv:
        return 'neg'


In [27]:
df = pyupbit.get_ohlcv("KRW-ETH", count=8000, interval="minute60")
df = pd.DataFrame(df)
df = df.drop(['value'], axis=1)

best_name = ""
best_result = pd.DataFrame()
best_profit = 0

for tweets in tweet_list:
    pd_tweets = pd.read_csv(tweets+'_tweets.csv')
    pd_tweets = pd_tweets.drop(['UserName', "Handle", "Likes", "Retweets", "Text"], axis=1)
    pd_tweets = pd_tweets.astype({'Comments' : 'string'})

    pd_tweets['Timestamp'] = pd_tweets['Timestamp'].apply(lambda x: datetime.strptime(x[:10] + ' ' + x[11:13] + ':00:00', '%Y-%m-%d %H:%M:%S'))
    pd_tweets
    
    #Creating new dataframe and new features
    tw_list = pd.DataFrame(pd_tweets)
    tw_list['text_processed'] = tw_list['Comments']

    #Removing RT, Punctuation etc
    remove_rt = lambda x: re.sub('RT @\w+: ',' ' ,str(x))
    rt = lambda x: re.sub('(@[A-Za-z0–9]+)|([^-9A-Za-z \t])|(\+:\/\/\S+)',' ' ,str(x))
    tw_list['text_processed'] = tw_list.text_processed.map(remove_rt).map(rt)
    tw_list['text_processed'] = tw_list.text_processed.str.lower()
    
    for index, row in tw_list['text_processed'].iteritems():
        score = SentimentIntensityAnalyzer().polarity_scores(row)
        neg = score['neg']
        neu = score['neu']
        pos = score['pos']
        comp = score['compound']

        if neu == 1.0:
            tw_list.loc[index, 'sentiment'] = 'neutral'
        elif neg > pos:
            tw_list.loc[index, 'sentiment'] = 'negative'
        else :
            tw_list.loc[index, 'sentiment'] = 'positive'
    #     if neg > pos:
    #         tw_list.loc[index, 'sentiment'] = 'negative'
    #     elif pos > neg:
    #         tw_list.loc[index, 'sentiment'] = 'positive'
    #     else:
    #         tw_list.loc[index, 'sentiment'] = 'neutral'

        tw_list.loc[index, 'neg'] = neg
        tw_list.loc[index, 'neu'] = neu
        tw_list.loc[index, 'pos'] = pos
        tw_list.loc[index, 'compound'] = comp
        
    #Creating new data frames for all sentiments (positive, negative and neutral)
    tw_list_negative = tw_list[tw_list['sentiment']=='negative']
    tw_list_positive = tw_list[tw_list['sentiment']=='positive']
    tw_list_neutral = tw_list[tw_list['sentiment']=='neutral']
    
    tw_list = tw_list[tw_list['sentiment'] != 'neutral']
    tw_group = tw_list.groupby('Timestamp').mean()
    
    tw_group['sentiment'] = tw_group.apply(sentiment_analysis, axis=1)

    result = pd.merge(df.reset_index().rename({'index' : 'Timestamp'}, axis=1), tw_group.reset_index(), how='left', on='Timestamp')[['Timestamp', 'open', 'close', 'sentiment']]
    
    result = result[result['open'].isna() == False]

    result = result.set_index('Timestamp')
    
    
    # 매수 매도 신호가 오면 close 가격에 매매했다고 가정

    # 초기금액 100으로 설정
    base = 100.0
    length = len(result)

    # 승패 기록. 매도시 수익률이 0보다 높으면 승, 아니면 패
    win = 0
    lose = 0

    # 트위터 데이터 중 처음 데이터의 시작일. 시장 수익률을 파악하기 위해 사용.
    first_day = 0

    # 매수상태인지 기록
    bought = False
    buy_price = 0

    for i in range(length):
        d = result.iloc[i]

        if bought:
            if d.sentiment == 'neg':
                base = base * (1 + (d.close - buy_price) / buy_price)
                bought = False
                print(f'{d.name} 매도 / 수익률 : {(d.close - buy_price) / buy_price * 100} \n')
                if ((d.close - buy_price) / buy_price) > 0 :
                    win += 1
                else :
                    lose += 1

            else:
                pass
        else:
            if d.sentiment == 'pos':
                buy_price = d.close
                bought = True
                print(f'{d.name} 매수')

                if first_day == 0 :
                    first_day = i
            else:
                pass
    print(f"{tweets} 분석 결과")
    print(f'최종 잔고 : {base:.2f}')
    print(f"총 매매 : {win+lose}, 승률 : {(win / (win+lose) * 100):.2f}")
    print(f"매매 시작일 : {result.iloc[first_day].name}")
    print(f"시장 수익률 : {(result.iloc[-1].close / result.iloc[first_day].close) * 100:.2f}%, 매매 수익률 : {base:.2f}% \n")
    print(best_profit, base / ((result.iloc[-1].close / result.iloc[first_day].close) * 100))
    
    if best_profit < (base / ((result.iloc[-1].close / result.iloc[first_day].close) * 100)) :
        best_name = tweets
        best_profit = (base / ((result.iloc[-1].close / result.iloc[first_day].close) * 100))
        best_result = result
    else:
        pass

2021-06-27 04:00:00 매수
2021-06-27 19:00:00 매도 / 수익률 : 1.292484442316898 

2021-06-28 07:00:00 매수
2021-06-28 11:00:00 매도 / 수익률 : 0.6208425720620843 

2021-06-28 19:00:00 매수
2021-07-01 00:00:00 매도 / 수익률 : 6.744487678339818 

2021-07-01 05:00:00 매수
2021-07-01 12:00:00 매도 / 수익률 : -2.269230769230769 

2021-07-01 13:00:00 매수
2021-07-03 19:00:00 매도 / 수익률 : 0.194325689856199 

2021-07-04 00:00:00 매수
2021-07-10 21:00:00 매도 / 수익률 : -3.249516441005803 

2021-07-11 03:00:00 매수
2021-07-11 07:00:00 매도 / 수익률 : 1.0492332526230832 

2021-07-11 09:00:00 매수
2021-07-12 08:00:00 매도 / 수익률 : 0.3193612774451098 

2021-07-12 12:00:00 매수
2021-07-14 21:00:00 매도 / 수익률 : -9.409448818897639 

2021-07-14 23:00:00 매수
2021-07-15 03:00:00 매도 / 수익률 : -1.2220817530552044 

2021-07-15 04:00:00 매수
2021-07-17 15:00:00 매도 / 수익률 : -4.269854824935952 

2021-07-17 17:00:00 매수
2021-07-19 18:00:00 매도 / 수익률 : -0.5371530886302597 

2021-07-20 05:00:00 매수
2021-07-20 10:00:00 매도 / 수익률 : -2.437902483900644 

2021-07-20 11:00:00 매수
202

2020-12-29 16:00:00 매수
2020-12-29 17:00:00 매도 / 수익률 : 0.6534305101784368 

2020-12-30 18:00:00 매수
2021-01-04 23:00:00 매도 / 수익률 : 46.906261578362354 

2021-01-06 11:00:00 매수
2021-01-07 19:00:00 매도 / 수익률 : 8.978583196046129 

2021-01-08 05:00:00 매수
2021-02-04 20:00:00 매도 / 수익률 : 22.21030042918455 

2021-02-09 02:00:00 매수
2021-02-12 21:00:00 매도 / 수익률 : 7.073238652186021 

2021-02-16 00:00:00 매수
2021-02-24 20:00:00 매도 / 수익률 : -3.9641943734015346 

2021-02-25 14:00:00 매수
2021-03-14 09:00:00 매도 / 수익률 : 19.93481803367735 

2021-03-14 10:00:00 매수
2021-05-22 22:00:00 매도 / 수익률 : 38.32116788321168 

2021-05-24 15:00:00 매수
2021-07-17 17:00:00 매도 / 수익률 : -15.474839197881195 

2021-07-25 13:00:00 매수
2021-08-08 14:00:00 매도 / 수익률 : 41.77215189873418 

2021-08-09 03:00:00 매수
2021-08-09 23:00:00 매도 / 수익률 : 3.1240960370263235 

2021-08-10 20:00:00 매수
brian_armstrong 분석 결과
최종 잔고 : 415.15
총 매매 : 11, 승률 : 81.82
매매 시작일 : 2020-12-29 16:00:00
시장 수익률 : 473.23%, 매매 수익률 : 415.15% 

0.8510584140192835 0.8772565489

2021-02-09 19:00:00 매도 / 수익률 : 40.844009042954035 

2021-02-11 20:00:00 매수
2021-02-19 15:00:00 매도 / 수익률 : 13.930742796722178 

2021-02-19 16:00:00 매수
2021-02-26 19:00:00 매도 / 수익률 : -20.250812819321876 

2021-03-01 09:00:00 매수
2021-03-03 11:00:00 매도 / 수익률 : 5.58743581999396 

2021-03-04 20:00:00 매수
2021-03-19 20:00:00 매도 / 수익률 : 18.549511854951188 

2021-03-19 21:00:00 매수
2021-03-24 13:00:00 매도 / 수익률 : -4.719207173194903 

2021-03-30 16:00:00 매수
2021-04-01 21:00:00 매도 / 수익률 : 9.425287356321839 

2021-04-02 16:00:00 매수
2021-05-19 19:00:00 매도 / 수익률 : 47.47393744987971 

2021-05-20 17:00:00 매수
2021-05-21 19:00:00 매도 / 수익률 : -1.2334801762114538 

2021-05-22 14:00:00 매수
2021-05-24 04:00:00 매도 / 수익률 : -13.171947312210753 

2021-05-24 15:00:00 매수
2021-06-04 12:00:00 매도 / 수익률 : 22.77714718123345 

2021-06-07 21:00:00 매수
2021-06-11 15:00:00 매도 / 수익률 : -12.750230982445334 

2021-06-14 13:00:00 매수
2021-06-23 18:00:00 매도 / 수익률 : -20.091004550227513 

2021-06-23 19:00:00 매수
2021-07-19 16:00:00 매도 / 

In [28]:
print(best_name)

blockchain


In [29]:
import plotly.graph_objects as go

fig = go.Figure()
# fig.add_trace(go.Scatter(x=result.index, y=result['open']))
fig.add_trace(go.Scatter(x=best_result.index, y=best_result['close'], mode='lines', name='none'))
fig.add_trace(go.Scatter(x=best_result[best_result['sentiment'] == 'neg'].index, y=best_result[best_result['sentiment'] == 'neg']['close'], mode='markers', name='neg'))
fig.add_trace(go.Scatter(x=best_result[best_result['sentiment'] == 'pos'].index, y=best_result[best_result['sentiment'] == 'pos']['close'], mode='markers', name='pos'))
fig.show()